This notebook generates a a cross-species browser whereby clicking on a point in the pig shows you the nearest connctopic vertex in the human and vice versa. This notebook requires HCP workbecnh to be installed and in your local path https://www.humanconnectome.org/software/get-connectome-workbench

In [ ]:
#### do our imports
import numpy as np
import nibabel as nib
import scipy.io as sio
from scipy import stats
import pandas as pd
import math
import h5py
import os,sys,argparse,re
import nilearn
import matplotlib.pyplot as plt
from IPython.display import display, HTML
import subprocess as sp
%matplotlib inline

In [ ]:
###### load all data
###### left hemisphere 
Lpig=h5py.File('./blueprints/L_pig27.mat','r')
variables=Lpig.items()
for var in variables:
    pig_name=var[0]
    pig_data=var[1]
    pig_data=np.array(pig_data).transpose()
    Lpig_data=pig_data
Lhum=h5py.File('./blueprints/L_hum27.mat','r')
variables=Lhum.items()
for var in variables:
    hum_name=var[0]
    hum_data=var[1]
    Lhum_data=np.array(hum_data).transpose()
    
    ######## load all data
Rpig=h5py.File('./blueprints/R_pig27.mat','r')
variables=Rpig.items()
for var in variables:
    pig_name=var[0]
    pig_data=var[1]
    pig_data=np.array(pig_data).transpose()
    Rpig_data=pig_data
Rhum=h5py.File('./blueprints/R_hum27.mat','r')
variables=Rhum.items()
for var in variables:
    hum_name=var[0]
    hum_data=var[1]
    Rhum_data=np.array(hum_data).transpose()
    
    

In [ ]:
##### define KL calculation
### Calculate the KL divergence as done in the Mars blueprint paper
def calc_kl(A,B):
    if type(A) == dict:
        A=list(A.values())
        A=np.array(A).transpose()
    else:
            pass
    if type(B) == dict:
        B=list(B.values())
        B=np.array(B).transpose()
    else:
        pass
    
    p=A.shape
    p=p[0]

    ## function takes two numpy arrays that are the blueprints not normalized
    def normalize(BP,p):
        BP[(np.isnan(BP))]=0
        row_sums=BP.sum(axis=1)
        BP=BP/row_sums[:,np.newaxis]
        return BP
    
    A=normalize(A,p)
    B=normalize(B,p)
    Amask= A!=0
    A_invmask=Amask!=1
    Bmask= B !=0
    B_invmask=Bmask!=1
 
    ##### pyhton implementation of Saad Jaabdi's matlab code fo KL divergence
    KL=np.dot(np.multiply(A,(np.log2((A+A_invmask)))),Bmask.transpose()) \
    - np.dot(A,(Bmask*np.log2(B+B_invmask)).transpose()) \
    + np.dot(Amask,(B*np.log2(B+B_invmask)).transpose())\
    - np.dot(Amask*np.log2(A+A_invmask),B.transpose())
    return KL,A,B

In [ ]:
### get KL's for cross species browser tool
###LEFT hemispehre
### cross species
L_HP=calc_kl(Lhum_data,Lpig_data)[0]
L_PH=calc_kl(Lpig_data,Lhum_data)[0]
### within hemisphere
L_HH=calc_kl(Lhum_data,Lhum_data)[0]
L_PP=calc_kl(Lpig_data,Lpig_data)[0]

#### RIGHT hemisphere
### cross species
R_HP=calc_kl(Rhum_data,Rpig_data)[0]
R_PH=calc_kl(Rpig_data,Rhum_data)[0]
### within hemisphere
R_HH=calc_kl(Rhum_data,Rhum_data)[0]
R_PP=calc_kl(Rpig_data,Rpig_data)[0]



In [ ]:
LH=np.hstack((L_HP,L_HH))
LP=np.hstack((L_PH,L_PP))
L=np.vstack((LH,LP))

RH=np.hstack((R_HP,R_HH))
RP=np.hstack((R_PH,R_PP))
R=np.vstack((RH,RP))
del L_HP,L_HH,L_PH,L_PP,R_HP,R_HH,R_PH,R_PP

In [ ]:
#### save the joint matrix 
sio.savemat('./Cross_species_browser/L_clicky.mat',mdict={'L': L})
sio.savemat('./Cross_species_browser/R_clicky.mat',mdict={'R': R})

In [ ]:
##### define clicky tool function
def clicky_tool(data,hemi,outdir,outname):
    #### create and empty gifti image
    out=f'{outdir}/{hemi}_{outname}'
    gi = nib.gifti.GiftiImage()
    da = nib.gifti.GiftiDataArray(np.float32(np.zeros(data.shape)), intent=0)
    gi.add_gifti_data_array(da)
    nib.save(gi,f'{out}.func.gii')
    #### first wb_commans
    cmd1=f'wb_command -cifti-create-dense-timeseries {out}.dtseries.nii -left-metric {out}.func.gii'
    print(cmd1)
    !{cmd1}
    print('running -cifti-correlation')
    cmd2=f'wb_command -cifti-correlation {out}.dtseries.nii {out}.dconn.nii'
    print(cmd2)
    !{cmd2}
    
    cft=nib.load(f'{out}.dconn.nii')
    im_obj=cft.header
    imdat = np.float32(data)
    new=nib.Cifti2Image(imdat,im_obj,cft.nifti_header)
    nib.save(new,f'{out}.dconn.nii')


clicky_tool(L,'L','./Cross_species_browser/','hum_pig_double_click')
clicky_tool(R,'R','./Cross_species_browser/','hum_pig_double_click')






In [ ]:
!rm ./Cross_species_browser/L_hum_pig_double_click.dtseries.nii
!rm ./Cross_species_browser/L_hum_pig_double_click.func.gii

!rm ./Cross_species_browser/R_hum_pig_double_click.dtseries.nii
!rm ./Cross_species_browser/R_hum_pig_double_click.func.gii

In [ ]:
!wb_command -add-to-spec-file L.CrossSpeciesBrowser.spec CORTEX_LEFT ./Cross_species_browser/L_hum_pig_double_click.dconn.nii

!wb_command -add-to-spec-file R.CrossSpeciesBrowser.spec CORTEX_RIGHT ./Cross_species_browser/R_hum_pig_double_click.dconn.nii